# Тестирование и отладка библиотек для работы с данными

In [65]:
# Загрузка блока инструкций для импорта библиотек из .py - файла
%run "load_big_libs.py"


In [66]:
# Генерация словаря шаблонов протоколов в папке data/intermid
def init_project_dictionaries():
    templates_csv = '../data/external/protocols_templates.csv'
    df_templates = pd.read_csv(templates_csv, sep=';')
    df_templates['protocols_df_name'] = df_templates['code'].apply(lambda x: x.replace('П', 'protocols_').replace('.', '_'))
    df_tmpl = df_templates.set_index('code')
    return df_tmpl    

In [67]:
# Инициализируем словарь шаблонов протоколов и смотрим, что получилось
df_templates = init_project_dictionaries()
df_templates.head()

,id,title,group,level,protocols_df_name
code,,,,,
П1.1,1,Базовый протокол «Распределение внимания учителя»,Распределение внимания учителя,1,protocols_1_1
П1.2,3,Усложненный протокол «Распределение внимания у...,Распределение внимания учителя,2,protocols_1_2
П1.3,2,Прицельный протокол «Распределение внимания уч...,Распределение внимания учителя,3,protocols_1_3
П2.1,4,Базовый протокол «Время на размышление»,Время на размышление,1,protocols_2_1
П2.2,6,Усложненный протокол «Время на размышление»,Время на размышление,2,protocols_2_2


In [68]:
# Функция позволяет загрузить все актуальные протоколы по проекту с сайта http://kurators.direktoria.org
# и сохранить протоколы каждого из проектов локально в папке data/external
# Одновременно инициализируются датасеты с протоколами по каждому из шаблонов
# В дальнейшем сохраненные версии датасетов могут скачиваться быстрее из локального хранилища
def load_protocols_data(df_templates):
    prot_mask_csv = 'http://kurators.direktoria.org/action.htm?action=export_data&do=get_template_protocols&filters=a:1:%7Bs:10:%22protocolId%22;s:%LEN_ID%:%22%ID%%22;%7D&format=csv&special=anonim'

    protocols_dfs = []

    for row in df_templates.iterrows():
        prot_id = str(row[1]['id'])
        prot_csv_link = prot_mask_csv.replace('%LEN_ID%', str(len(prot_id))).replace('%ID%', prot_id)
        print(f'Загружаем протокол {row[1]["title"]}')
        prot_df_name = row[1]['protocols_df_name']
        globals()[prot_df_name] = pd.read_csv(prot_csv_link, sep=';')
        globals()[prot_df_name].to_csv('../data/external/' + prot_df_name + '_raw.csv', sep=';')

In [69]:
# Функция позволяет загрузить ранее сохраненные "сырые" протоколы по проекту 
# Инициализируются датасеты с протоколами по каждому из шаблонов
def load_raw_protocols_data(df_templates):
    protocols_dfs = []

    for row in df_templates.iterrows():
        print(f'Загружаем протокол {row[1]["title"]}')
        prot_df_name = row[1]['protocols_df_name']
        globals()[prot_df_name] = pd.read_csv('../data/external/' + prot_df_name + '_raw.csv', sep=';')

In [73]:
load_protocols_data(df_templates)

Загружаем протокол Базовый протокол «Распределение внимания учителя»
Загружаем протокол Усложненный протокол «Распределение внимания учителя»
Загружаем протокол Прицельный протокол «Распределение внимания учителя»
Загружаем протокол Базовый протокол «Время на размышление»
Загружаем протокол Усложненный протокол «Время на размышление»
Загружаем протокол Прицельный протокол «Время на размышление»
Загружаем протокол Базовый протокол «Тип вопросов»
Загружаем протокол Усложненный протокол «Тип вопросов»
Загружаем протокол Прицельный протокол «Тип вопросов»
Загружаем протокол Базовый протокол «Качество обратной связи»
Загружаем протокол Усложненный протокол «Качество обратной связи»
Загружаем протокол Прицельный протокол «Качество обратной связи»
Загружаем протокол Базовый протокол «Время речи учеников»
Загружаем протокол Усложненный протокол «Время речи учеников»
Загружаем протокол Прицельный протокол «Время речи учеников»
Загружаем протокол Базовый протокол «Правила речи»
Загружаем протоко

In [29]:
#load_protocols_data(df_tmpl)
#print(df_templates.head())
#protocols_2_1.rename(columns = {'Unnamed: 0': 'protocol_id'}, inplace = True)
protocols_2_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 55 columns):
 #   Column                                                                                                                                           Non-Null Count  Dtype  
---  ------                                                                                                                                           --------------  -----  
 0   Id протокола                                                                                                                                     216 non-null    int64  
 1   Код шаблона                                                                                                                                      216 non-null    object 
 2   Название шаблона                                                                                                                                 216 non-null    object 
 3   Дата заполнения     

In [19]:
# Блок для тестирования функции загрузки протоколов из локального хранилища
# load_raw_protocols_data(df_tmpl)
# print(protocols_6_2.info())

### Функция унификации датасетов по каждому из шаблонов протоколов 
- Очищенные от лишних данных датасеты сохраняются в папке data/intermid
- Проводятся следующие преобразования:
- Удаляются вторичные колонки
- Удаляются личные данные
- Заполняются дополнительные колонки с фиксированными названиями параметров.
- Если данные для колонки отсутствуют, устанавливается значение NaN

После унификации датасетов по каждому из шаблонов протоколов они могут быть объединены в единый датасет, который и будет далее использоваться для анализа.

Общие мета-колонки для каждого протокола
- prot_id (Id протокола)
- template_code (Код шаблона)
- date_of_lesson (Дата заполнения)
- org_id (Id организации)
- teacher_id (Id учителя, который провел урок)
- observer_id (Id учителя, который наблюдал за уроком)
- curator_id (Id куратора пары учителей)

Колонки с данными
- class (Параллель класса, в котором проводился урок. Число от 1 до 11)
- satisfaction (Уровень удовлетворенности учителя проведенным уроком. Число от 1 до 4)
- wish_to_change (Желание что-то изменить на следующих уроках. Число от 1 до 4)

- pupils_all (Количество учеников на уроке)
- pupils_strong (Количество сильных учеников на уроке)
- pupils_middle (Количество средних учеников на уроке)
- pupils_weak (Количество слабых учеников на уроке)

- interactions_all (Количество взаимодействий с учениками на уроке)
- interactions_strong (Количество взаимодействий с сильными учениками на уроке)
- interactions_middle (Количество взаимодействий с средними учениками на уроке)
- interactions_weak (Количество взаимодействий со слабыми учениками на уроке)
    
- attention_all (Количество учеников, с которыми взаимодействовал учитель)
- attention_strong (Количество сильных учеников, с которыми взаимодействовал учитель) 
- attention_middle (Количество средних учеников, с которыми взаимодействовал учитель)
- attention_weak (Количество слабых учеников, с которыми взаимодействовал учитель)
    
- teacher_comments ()


In [78]:
# Функция чистки датасетов по каждому из шаблонов протоколов 
# Очищенные от лишних данных датасеты сохраняются в папке data/intermid
# Проводятся следующие преобразования:
# Удаляются вторичные колонки
# Удаляются личные данные
# Заполняются дополнительные колонки с фиксированными названиями параметров.
# Если данные для колонки отсутствуют, устанавливается значение NaN

cols_substitutes = {
    'class': ['Параллель класса'],
    'satisfaction': ['Удовлетворенность уроком'],
    'wish_to_change': ['Необходимость изменений'],
    
    'pupils_all': ['Количество учеников на уроке'],
    'pupils_strong': ['Сколько учеников из каждой группы было на уроке::Сильные', 
                    'Количество учеников на уроке::Сильные'],
    'pupils_middle': ['Сколько учеников из каждой группы было на уроке::Средние', 
                    'Количество учеников на уроке::Средние'],
    'pupils_weak': ['Сколько учеников из каждой группы было на уроке::Слабые', 
                  'Количество учеников на уроке::Слабые'],

    'interactions_all': ['Количество ответов на уроке'],
    'interactions_strong': ['Сколько всего взаимодействий было с учениками::Сильные',
                           'Количество ответов на уроке::Сильные',
                           'Количество ответов на уроке от::Сильные'],
    'interactions_middle': ['Сколько всего взаимодействий было с учениками::Средние',
                           'Количество ответов на уроке::Средние',
                           'Количество ответов на уроке от::Средние'],
    'interactions_weak': ['Сколько всего взаимодействий было с учениками::Слабые',
                           'Количество ответов на уроке::Слабые',
                           'Количество ответов на уроке от::Слабые'],
    
    'attention_all': ['Количество отвечавших учеников'],
    'attention_strong': ['Сколько учеников были в зоне внимания учителя::Сильные',
                        'Сколько учеников из каждой группы отвечали::Сильные',
                        'Количество отвечавших учеников::Сильные'],
    'attention_middle': ['Сколько учеников были в зоне внимания учителя::Средние',
                        'Сколько учеников из каждой группы отвечали::Средние',
                        'Количество отвечавших учеников::Средние'],
    'attention_weak': ['Сколько учеников были в зоне внимания учителя::Слабые',
                        'Сколько учеников из каждой группы отвечали::Слабые',
                        'Количество отвечавших учеников::Слабые'],
    
    
    'teacher_comments': ['Выводы и идеи (Вы удовлетворены? Нужно ли что-то изменить? Какую достижимую цель вы хотели бы себе поставить? Понимаете ли вы, как ее достичь?)']
}   


def build_all_protocols_data(df_templates):
    dfs = []

    for row in df_templates.iterrows():
        print(f'Обрабатываем протокол {row[1]["title"]}')
        prot_df_name = row[1]['protocols_df_name']
        cols = ['Id протокола', 'Код шаблона', 'Дата заполнения', 'Id организации', 
                'Id учителя', 'Id наблюдателя', 'Id куратора']
        new_cols = ['prot_id', 'template_code', 'date_of_lesson', 'org_id', 
                    'teacher_id', 'observer_id', 'curator_id']
        df = globals()[prot_df_name].loc[:, cols]
        df.columns = new_cols
        for cs in cols_substitutes.items():
            old_col = pd.Series(np.nan)
            for raw_col in cs[1]:
                if raw_col in globals()[prot_df_name].columns:
                    old_col = globals()[prot_df_name][raw_col]
            df[cs[0]] = old_col        
        #df.set_index('prot_id', inplace=True)
        df.to_csv('../data/intermid/' + prot_df_name + '_stand.csv', sep=';')
        dfs.append(df)
        
    df_full = pd.concat(dfs, ignore_index=True)
    df_full.to_csv('../data/intermid/full_df_stand.csv', sep=';')  
    return df_full

In [79]:
df_protocols = build_all_protocols_data(df_templates)

Обрабатываем протокол Базовый протокол «Распределение внимания учителя»
Обрабатываем протокол Усложненный протокол «Распределение внимания учителя»
Обрабатываем протокол Прицельный протокол «Распределение внимания учителя»
Обрабатываем протокол Базовый протокол «Время на размышление»
Обрабатываем протокол Усложненный протокол «Время на размышление»
Обрабатываем протокол Прицельный протокол «Время на размышление»
Обрабатываем протокол Базовый протокол «Тип вопросов»
Обрабатываем протокол Усложненный протокол «Тип вопросов»
Обрабатываем протокол Прицельный протокол «Тип вопросов»
Обрабатываем протокол Базовый протокол «Качество обратной связи»
Обрабатываем протокол Усложненный протокол «Качество обратной связи»
Обрабатываем протокол Прицельный протокол «Качество обратной связи»
Обрабатываем протокол Базовый протокол «Время речи учеников»
Обрабатываем протокол Усложненный протокол «Время речи учеников»
Обрабатываем протокол Прицельный протокол «Время речи учеников»
Обрабатываем протокол Б

In [80]:
df_protocols.describe(include = 'all')

,prot_id,template_code,date_of_lesson,org_id,teacher_id,observer_id,curator_id,class,satisfaction,wish_to_change,...,pupils_weak,interactions_all,interactions_strong,interactions_middle,interactions_weak,attention_all,attention_strong,attention_middle,attention_weak,teacher_comments
count,2973.000000,2973,2973,2973.000000,2973.000000,2973.000000,2973.000000,2973.000000,2973.000000,2973.000000,...,2041.00000,1742.000000,1808.000000,1808.000000,1808.000000,1928.000000,2041.000000,2041.000000,2041.000000,2863
unique,NaN,19,183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2693
top,NaN,П1.2,26.09.2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,удовлетворена
freq,NaN,278,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
mean,1564.918264,NaN,NaN,102.654894,714.004709,714.048772,707.141608,1.558022,1.160108,0.674067,...,3.11612,16.095293,4.474558,5.674779,3.548673,6.627075,2.748163,3.529152,2.333170,NaN
std,881.890611,NaN,NaN,23.745439,137.544724,137.640512,134.957220,2.813831,1.679761,1.136565,...,2.53867,20.134958,7.185071,9.074212,6.480558,6.681379,2.806204,3.067422,2.276634,NaN
min,41.000000,NaN,NaN,1.000000,472.000000,472.000000,471.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,798.000000,NaN,NaN,87.000000,597.000000,597.000000,591.000000,0.000000,0.000000,0.000000,...,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,NaN
50%,1561.000000,NaN,NaN,108.000000,721.000000,721.000000,713.000000,0.000000,0.000000,0.000000,...,3.00000,11.000000,0.000000,0.000000,0.000000,6.000000,2.000000,3.000000,2.000000,NaN
75%,2335.000000,NaN,NaN,121.000000,827.000000,827.000000,819.000000,2.000000,3.000000,1.000000,...,4.00000,25.000000,7.000000,9.000000,5.000000,10.000000,4.000000,5.000000,3.000000,NaN


In [85]:
# Получаем информацию по количеству протоколов для каждого из шаблонов
df_protocols.template_code.value_counts()

П1.2    278
П3.2    243
П2.2    216
П4.2    201
П1.3    190
П3.3    183
П1.1    182
П3.1    173
П2.3    168
П4.1    166
П2.1    148
П4.3    143
П5.2    131
П6.2    112
П5.3    102
П5.1     95
П6.1     91
П7.1     77
П6.3     74
Name: template_code, dtype: int64

In [82]:
df_protocols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2973 entries, 0 to 2972
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   prot_id              2973 non-null   int64  
 1   template_code        2973 non-null   object 
 2   date_of_lesson       2973 non-null   object 
 3   org_id               2973 non-null   int64  
 4   teacher_id           2973 non-null   int64  
 5   observer_id          2973 non-null   int64  
 6   curator_id           2973 non-null   int64  
 7   class                2973 non-null   int64  
 8   satisfaction         2973 non-null   int64  
 9   wish_to_change       2973 non-null   int64  
 10  pupils_all           2973 non-null   int64  
 11  pupils_strong        2041 non-null   float64
 12  pupils_middle        2041 non-null   float64
 13  pupils_weak          2041 non-null   float64
 14  interactions_all     1742 non-null   float64
 15  interactions_strong  1808 non-null   f

In [83]:
df_protocols.head()

,prot_id,template_code,date_of_lesson,org_id,teacher_id,observer_id,curator_id,class,satisfaction,wish_to_change,...,pupils_weak,interactions_all,interactions_strong,interactions_middle,interactions_weak,attention_all,attention_strong,attention_middle,attention_weak,teacher_comments
0,816,П1.1,01.02.2022,133,726,725,724,0,0,0,...,NaN,30.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,Больше внимания уделять слабым ученикам и моти...
1,1162,П1.1,02.02.2022,128,736,735,734,0,0,0,...,NaN,18.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,Учитель уделяет внимание всем учащимся
2,1170,П1.1,02.02.2022,128,735,736,734,0,0,0,...,NaN,17.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,"Уделять внимание не только тем, кто поднимает ..."
3,1069,П1.1,04.02.2022,133,725,726,724,0,0,0,...,NaN,39.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,Больше внимания уделять слабым ученикам и моти...
4,295,П1.1,07.02.2022,79,534,533,532,0,0,0,...,NaN,18.0,NaN,NaN,NaN,11.0,NaN,NaN,NaN,Своё внимание учитель распределял на всех обу...


In [35]:
cols_substitutes = {
  'pupils_all': ['Количество учеников на уроке'],
  'pupils_strong': ['Сколько учеников из каждой группы было на уроке::Сильные', 
                    'Количество учеников на уроке::Сильные'],
  'pupils_middle': ['Сколько учеников из каждой группы было на уроке::Средние', 
                    'Количество учеников на уроке::Средние'],
  'pupils_weak': ['Сколько учеников из каждой группы было на уроке::Слабые', 
                  'Количество учеников на уроке::Слабые']
}  

cols = ['Id протокола', 'Код шаблона', 'Дата заполнения', 'Id организации', 'Id учителя', 'Id наблюдателя', 'Id куратора']
df_tmp = protocols_6_2.loc[:, cols]
df_tmp.columns = ['prot_id', 'template_code', 'date_of_lesson', 'org_id', 'teacher_id', 'observer_id', 'curator_id']
for cs in cols_substitutes.items():
    old_col = pd.Series(np.nan)
    for raw_col in cs[1]:
        if raw_col in protocols_6_2.columns:
            old_col = protocols_6_2[raw_col]
    df_tmp[cs[0]] = old_col        

df_tmp.head()

,prot_id,template_code,date_of_lesson,org_id,teacher_id,observer_id,curator_id,pupils_all,pupils_strong,pupils_middle,pupils_weak
0,162,П6.2,13.04.2022,111,796,795,794,19,6,8,5
1,166,П6.2,13.04.2022,111,795,796,794,19,7,10,2
2,426,П6.2,10.05.2022,98,632,633,631,3,0,2,1
3,1613,П6.2,11.05.2022,100,687,688,689,18,6,8,4
4,1209,П6.2,11.05.2022,128,735,736,734,9,4,3,2


In [70]:
#print(protocols_6_2.columns.get_loc('Unnamed'))
def drop_postproc_cols():
    unnamed_ind = False
    for col in protocols_6_2.columns:
        if 'Unnamed' in col:
            unnamed_ind = int(col.replace('Unnamed: ', ''))
            break;
    if unnamed_ind:
        cols = [x for x in range(unnamed_ind)]
        protocols_6_2_initial = protocols_6_2.iloc[:, cols]    
    else:
        protocols_6_2.copy()

#protocols_6_2_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 41 columns):
 #   Column                                                                                                                                           Non-Null Count  Dtype 
---  ------                                                                                                                                           --------------  ----- 
 0   Код шаблона                                                                                                                                      83 non-null     object
 1   Название шаблона                                                                                                                                 83 non-null     object
 2   Дата заполнения                                                                                                                                  83 non-null     object
 3   Дата обсуждения            